In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv("clickbait_data.csv")

Index(['headline', 'clickbait'], dtype='object')

'Should I Get Bings'

In [7]:
cv = CountVectorizer(ngram_range=(1, 3)).fit(data.headline)

In [9]:
print("Vocabulary size: {}".format(len(cv.vocabulary_))) 
# print("Vocabulary:\n{}".format(cv.get_feature_names()))

Vocabulary size: 22761


In [52]:
X_train = data.headline[0:int(0.8*len(data.headline))]
X_test = data.headline[int(0.8*len(data.headline)):]

In [43]:
y_train = data.clickbait[0:int(0.8*len(text))]
y_test = data.clickbait[int(0.8*len(text)):]

In [57]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=1000))
param_grid = {"logisticregression__C": [0.001, 0.01, 0.1, 1, 10, 100],
                  "tfidfvectorizer__ngram_range": [(1, 1), (1, 2), (1, 3)]}

In [58]:
# param_grid = {"logisticregression__C": [100],
#                   "tfidfvectorizer__ngram_range": [(1, 1)]}

In [59]:
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_)) 
print("Best parameters:\n{}".format(grid.best_params_))

Best cross-validation score: 0.97
Best parameters:
{'logisticregression__C': 10, 'tfidfvectorizer__ngram_range': (1, 2)}


# fitting a model not using cross validation

In [61]:
text = cv.transform(data.headline).toarray().tolist()
text_train = text[0:int(0.8*len(text))]
text_test = text[int(0.8*len(text)):]

In [33]:
model = LogisticRegression()
model.fit(text_train, y_train)

LogisticRegression()

In [37]:
y_pred = model.predict_proba(text_train)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_train)

0.994296875

In [40]:
y_pred_test = model.predict(text_test)
accuracy_score(y_pred_test, y_test)

0.9675

# stemming

In [64]:
import spacy
import nltk

/Users/gary/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
en_nlp = spacy.load('en')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.